In [1]:
from helpers.setup import setup_django

In [2]:
setup_django()

In [3]:
import pandas as pd
import pypandoc

from django.db.models import F, Q, Prefetch

from signprot.models import SignprotComplex
from protein.models import Protein, ProteinConformation

from Bio.PDB import Selection, PDBParser
from Bio.PDB.NeighborSearch import NeighborSearch

from contactnetwork.interaction import *
from contactnetwork.pdb import *
from contactnetwork.models import *
from io import StringIO

from protein.models import ProteinConformation

from structure.models import Structure

from signprot.models import SignprotComplex

import copy


# Reverse Lookup of Residues that belong to a ProteinConformation

In [4]:
ProteinConformation.objects.get(protein__entry_name="6cmo_a").residue_set.all()

<QuerySet [<Residue: M1>, <Residue: G2>, <Residue: C3>, <Residue: T4>, <Residue: L5>, <Residue: S6>, <Residue: A7>, <Residue: E8>, <Residue: D9>, <Residue: K10>, <Residue: A11>, <Residue: A12>, <Residue: V13>, <Residue: E14>, <Residue: R15>, <Residue: S16>, <Residue: K17>, <Residue: M18>, <Residue: I19>, <Residue: D20>, '...(remaining elements truncated)...']>

# Compute Interactions Prep

In [7]:
pdb_name='6CMO'

do_distances = True
do_interactions = True
do_complexes = True
distances = []
classified = []
classified_complex = []

# Ensure that the PDB name is lowercase
pdb_name = pdb_name.lower()

# Get the pdb structure
struc = Structure.objects.get(protein_conformation__protein__entry_name=pdb_name)
pdb_io = StringIO(struc.pdb_data.pdb)
# Get the preferred chain
preferred_chain = struc.preferred_chain.split(',')[0]

# Get the Biopython structure for the PDB
s = PDBParser(PERMISSIVE=True, QUIET=True).get_structure('ref', pdb_io)[0]
#s = pdb_get_structure(pdb_name)[0]
chain = s[preferred_chain]
#return classified, distances

# remove residues without GN and only those matching receptor.
residues = struc.protein_conformation.residue_set.exclude(generic_number=None).all().prefetch_related('generic_number')
dbres = {}
dblabel = {}
for r in residues:
    dbres[r.sequence_number] = r
    dblabel[r.sequence_number] = r.generic_number.label
ids_to_remove = []
for res in chain:
    if not res.id[1] in dbres.keys() and res.get_resname() != "HOH":
        ids_to_remove.append(res.id)
for i in ids_to_remove:
    chain.detach_child(i)

In [8]:
complex = SignprotComplex.objects.get(structure=struc)

# Get all GPCR residue atoms based on preferred chain
gpcr_atom_list = [ atom for residue in Selection.unfold_entities(s[preferred_chain], 'R') if is_aa(residue) \
                for atom in residue.get_atoms()]

# Get all residue atoms from the coupled protein (e.g. G-protein)
# NOW: select alpha subnit protein chain using complex model
sign_atom_list = [ atom for residue in Selection.unfold_entities(s[complex.alpha], 'R') if is_aa(residue) \
                    for atom in residue.get_atoms()]

ns_gpcr = NeighborSearch(gpcr_atom_list)
ns_sign = NeighborSearch(sign_atom_list)

# For each GPCR atom perform the neighbor search on the signaling protein
all_neighbors = {(gpcr_atom.parent, match_res) for gpcr_atom in gpcr_atom_list
                for match_res in ns_sign.search(gpcr_atom.coord, 4.5, "R")}

# For each pair of interacting residues, determine the type of interaction
residues_sign = ProteinConformation.objects.get(protein__entry_name=pdb_name+"_"+complex.alpha.lower()).residue_set.exclude(generic_number=None).all().prefetch_related('generic_number')

# grab labels from sign protein
dbres_sign = {}
dblabel_sign = {}
for r in residues_sign:
    dbres_sign[r.sequence_number] = r
    dblabel_sign[r.sequence_number] = r.generic_number.label

# Find interactions
interactions = [InteractingPair(res_pair[0], res_pair[1], dbres[res_pair[0].id[1]], dbres_sign[res_pair[1].id[1]], struc) for res_pair in all_neighbors if res_pair[0].id[1] in dbres and res_pair[1].id[1] in dbres_sign ]

# Filter unclassified interactions
classified_complex = [interaction for interaction in interactions if len(interaction.get_interactions()) > 0]

# Convert to dictionary for water calculations
interaction_pairs = {}
for pair in classified_complex:
    res_1 = pair.get_residue_1()
    res_2 = pair.get_residue_2()
    key =  res_1.get_parent().get_id()+str(res_1.get_id()[1]) + "_" + res_2.get_parent().get_id()+str(res_2.get_id()[1])
    interaction_pairs[key] = pair

In [20]:
interactions[7].get_interactions()